In [1]:
# todo:
# - understand how the training data text is represented for chain of thought ot 3SUM sequences
# - function for evaluating accuracy
# - understand decoder implementation
# - function to print rank-n for layer m and then iterate over using this


# experiments:
# - write the claims of the paper: COT is uninterpretable as it can be hidden, but this hidden COT still increases performs
# - see where it is being replaced with '.' and where it is just the numbers
# - check this across layer, across rank
# - is this gradual or not gradual?
# - check if these decoded sequences from different ranks and layers are in anyway simialr to the actual trianing dataset.
# - write the punchline: you only see the decoded text of COT, but not what is there in the rest of the logits as information.

In [159]:
from src.match3 import *
from src.utils import InputEmbedCausalTransformer
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import Levenshtein
import json
import numpy as np
import os 
import pandas as pd
import torch
import torch.nn.functional as F

In [3]:
MODEL_PATH = "/home/loki/projects/filler_tokens/output_dir/2024-08-14-22-matchdata-checkpoint-final/model_weights.pt"
CONFIG_FILE = "/home/loki/projects/filler_tokens/misc/llama_d384l4h6.json"

In [4]:
train_df = pd.read_csv('data/matchdata_trainset_2024-08-13.csv', header=None, names=["text"])
test_df = pd.read_csv('data/matchdata_testset_2024-08-13.csv', header=None, names=["text"])

In [5]:
test_set = Match3VectorDataset(test_df, 3, 10, 10, 'P')
print(test_set.input_dim)

validate encodings
raw input 0  433 450 421 129 107 924 489 711 540 034 P 0- 3 0- 4 3- 2 4- 0 5- 5 0- 8 7- 1 0- 7 9- 4 1- 8 3- 5 1- 7 1- 3 6- 8 1- 1 8- 9 9- 4 3- 5 4- 2 5- 5 6- 8 2- 2 2- 6 9- 5 3- 2 5- 0 6- 5 3- 0 8- 9 9- 5 4- 2 6- 8 7- 1 8- 4 4- 3 6- 3 5- 6 8- 4 9- 8 7- 9 8- 9 6- 1 7- 2 7- 7 9- 7 9- 5 A False
encoded sample 0 {'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16), 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    5,   28,
           5,   29,    8,   27,    9,   25,   10,   30,    5,   33,   12,   26,
           5,   32,   14,   29,    6,   33,    8,   30,    6,   32,    6,   28,
          11,   33,    6,   26,   13,   34,   14,   29,    8,   30,    9,   27,
          10,   30,   11,   33,    7,   27,    7,  

In [6]:
test_set.tensorize_inputs_worker(pd.Series({"text":test_df.iloc[-1]}))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float16)

In [7]:
def string_to_input_tensors(input_string):
    return {"input_ids":test_set.tensorize_inputs_worker({"text":pd.Series([input_string], index=['text'], name='1999')}).squeeze()}

In [8]:
string_to_input_tensors("1 P A")

{'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16)}

In [9]:
def string_to_label_tensors(input_string):
    return {"labels":test_set.tensorize_labels_worker({"text":pd.Series([input_string], index=['text'], name='1999')})}

In [10]:
string_to_input_tensors("100 P A")

{'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16)}

In [197]:
class BlockOutputWrapper(torch.nn.Module):
    def __init__(self, block, unembed_matrix, norm):
        super().__init__()
        self.block = block
        self.unembed_matrix = unembed_matrix
        self.norm = norm
        self.block_output_unembedded = None

    def forward(self, *args, **kwargs):
        output = self.block(*args, **kwargs)
        if isinstance(output, tuple):
            self.block_output_unembedded = self.unembed_matrix(self.norm(output[0]))
            return output
        else:
            self.block_output_unembedded = self.unembed_matrix(self.norm(output))
            return output

    def reset_block_output(self):
        self.block_output_unembedded = None

class LlamaHelper:
    def __init__(self, config_file, model_path, test_set):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        config = AutoConfig.from_pretrained(config_file)
        model = InputEmbedCausalTransformer(AutoModelForCausalLM.from_config(config), test_set.input_dim)
        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict, strict=False)
        model = model.to(self.device)
        self.model = model
        self.word_index_map = test_set.word_index_map
        self.data_len = test_set.data_len
        self.mod = test_set.mod
        self.input_dim = test_set.input_dim
        
        for i, layer in enumerate(self.model.base_model.model.layers):
            self.model.base_model.model.layers[i] = BlockOutputWrapper(layer, self.model.base_model.lm_head, self.model.base_model.model.norm)

    def decode_tensors(self, sequence):
        decoded_sequence = []
        marker_found = False

        for token in sequence:
            token = token.item()
            if token == -100:
                if not marker_found:
                    decoded_sequence.append("[MASK]")
                continue
            elif token == 0:
                decoded_sequence.append("[EOS]")
                break  # Stop decoding after EOS
            elif token < len(self.word_index_map):
                # Regular word
                word = list(self.word_index_map.keys())[list(self.word_index_map.values()).index(token)]
                decoded_sequence.append(word)
                if word in ["A", "P"]:
                    marker_found = True
            else:
                # Handle digit labels
                offset = len(self.word_index_map)
                if token < offset + self.data_len * 2:
                    # Tuple index encoding
                    idx = (token - offset) % self.data_len
                    tuple_pos = (token - offset) // self.data_len
                    decoded_sequence.append(f"{tuple_pos}-{idx}")
                else:
                    # Single digit or digit in tuple
                    char_pos = (token - offset - self.data_len * 2) // self.mod
                    digit = (token - offset - self.data_len * 2) % self.mod
                    if char_pos == 0 or len(decoded_sequence) == 0 or not decoded_sequence[-1][-1].isdigit():
                        decoded_sequence.append(str(digit))
                    else:
                        decoded_sequence[-1] += str(digit)

        return " ".join(decoded_sequence)


    def set_add_attn_output(self, layer, add_output):
        self.model.base_model.model.layers[layer].attn_add_tensor(add_output)

    def get_attn_activations(self, layer):
        return self.model.base_model.model.layers[layer].get_attn_activations()

    def reset_all_layers(self):
        for layer in self.model.base_model.model.layers:
            layer.reset_block_output()
            
    @staticmethod
    def get_tokens(model, layer_idx, input_ids, decode_tensors, num_layers, rank=1, device="cuda", skip_idx=None):
        with torch.no_grad():
            outputs = model(input_ids.float().unsqueeze(0))
            logits = outputs.logits

        last_token_logits = logits[0, -1, :]

        if layer_idx < num_layers:
            layer = model.base_model.model.layers[layer_idx]
            if layer.block_output_unembedded is not None:
                last_token_logits = layer.block_output_unembedded[0, -1, :]

        # Get top k+1 values and indices, where k is max(rank, 1)
        k = max(rank, 1) + 1
        val, idx = torch.topk(last_token_logits, k)

        # If skip_idx is provided and matches the top prediction, use the next best
        if skip_idx is not None and idx[0].item() == skip_idx:
            selected_idx = 1
        else:
            selected_idx = 0

        # Ensure we don't go out of bounds if rank is 1
        selected_idx = min(selected_idx, len(idx) - 1)

        token = decode_tensors(idx[selected_idx].unsqueeze(-1)).strip()

        return token, idx[selected_idx].item()


    def create_new_token_input(self, token_id):
        new_input = torch.zeros(1, self.input_dim, dtype=torch.float16)
        if token_id < len(self.word_index_map):
            new_input[0, token_id] = 1
        else:
            # Handle digit sequences
            offset = len(self.word_index_map)
            if token_id < offset + self.data_len * 2:
                # Tuple index encoding
                idx = (token_id - offset) % self.data_len
                tuple_pos = (token_id - offset) // self.data_len
                new_input[0, offset + tuple_pos * self.data_len + idx] = 1
            else:
                # Single digit or digit in tuple
                char_pos = (token_id - offset - self.data_len * 2) // self.mod
                digit = (token_id - offset - self.data_len * 2) % self.mod
                new_input[0, offset + self.data_len * 2 + char_pos * self.mod + digit] = 1
        return new_input
    

    def print_logit_progression(self, inputs, max_new_tokens=len(test_set[0]['labels']), layer_number=None, rank=1, skip_idx = None):
        

        self.reset_all_layers()
        num_layers = len(self.model.base_model.model.layers)
        result_dict = {f"h{i}_out": [] for i in range(num_layers + 1)}
        result_dict["h_out"] = []
        input_ids = inputs['input_ids'].to(self.device)
        generated_sequence = input_ids.clone()

        for _ in range(max_new_tokens):
            self.reset_all_layers()
            
            if layer_number is not None:
                if layer_number > num_layers:
                    print(f"Error: Layer number {layer_number} is out of range. Max layer is {num_layers}.")
                    return {}
                token, token_id = self.get_tokens(self.model, layer_number, generated_sequence, self.decode_tensors, num_layers, rank, self.device, skip_idx)
                layer_name = f"h{layer_number}_out" if layer_number < num_layers else "h_out"
                result_dict[layer_name].append(token)
            else:
                for i in range(num_layers + 1):
                    token, token_id = self.get_tokens(self.model, i, generated_sequence, self.decode_tensors, num_layers, rank, self.device, skip_idx)
                    layer_name = f"h{i}_out" if i < num_layers else "h_out"
                    result_dict[layer_name].append(token)
                    
            if token == "[EOS]": break
                

            try:
                new_token_input = self.create_new_token_input(token_id).to(self.device)
                    
                generated_sequence = torch.cat([generated_sequence, new_token_input], dim=0)
            except:
                generated_sequence = torch.cat([generated_sequence], dim=0)
                break


        # Print results
        if layer_number is not None:
            layer_name = f"h{layer_number}_out" if layer_number < num_layers else "h_out"
            print(f"{rank}th highest logit for {layer_name}:")
            print(" ".join(result_dict[layer_name]))
        else:
            print(f"{rank}th highest logit:")
            for layer_name, tokens in result_dict.items():
                print(f"{layer_name:<5}: " + " ".join(tokens))

        return result_dict
    
    def get_layer_logits(self, inputs, layer_idx):
        self.reset_all_layers()
        num_layers = len(self.model.base_model.model.layers)
        
        if layer_idx > num_layers:
            raise ValueError(f"Error: Layer number {layer_idx} is out of range. Max layer is {num_layers}.")
        
        input_ids = inputs['input_ids'].to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_ids.float().unsqueeze(0))
            logits = outputs.logits
        
        if layer_idx < num_layers:
            layer = self.model.base_model.model.layers[layer_idx]
            if layer.block_output_unembedded is not None:
                logits = layer.block_output_unembedded
        
        return logits


In [198]:
model = LlamaHelper(CONFIG_FILE, MODEL_PATH, test_set)

In [202]:
model.print_logit_progression(test_set[50], rank=1,skip_idx=3)

1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] [EOS]
h1_out: 2 2 [EOS] [EOS] [EOS]
h2_out: 2 2 [EOS] [EOS] [EOS]
h3_out: False False False False [EOS]
h4_out: 
h_out: False False False False [EOS]


{'h0_out': ['[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]'],
 'h1_out': ['2', '2', '[EOS]', '[EOS]', '[EOS]'],
 'h2_out': ['2', '2', '[EOS]', '[EOS]', '[EOS]'],
 'h3_out': ['False', 'False', 'False', 'False', '[EOS]'],
 'h4_out': [],
 'h_out': ['False', 'False', 'False', 'False', '[EOS]']}

In [203]:
for idx in range(1,100):
    model.print_logit_progression(test_set[idx], rank=1, skip_idx=3)

1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] [EOS]
h1_out: 2 2 2 [EOS] [EOS]
h2_out: A [EOS] A [EOS] [EOS]
h3_out: False False False False [EOS]
h4_out: 
h_out: False False False False [EOS]
1th highest logit:
h0_out: [EOS] False [EOS] [EOS] [EOS]
h1_out: A False [EOS] [EOS] [EOS]
h2_out: A False [EOS] [EOS] [EOS]
h3_out: A False False False [EOS]
h4_out: 
h_out: A False False False [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS]
h1_out: [EOS] 0-6 False [EOS]
h2_out: [EOS] A A [EOS]
h3_out: 9 A True [EOS]
h4_out: 
h_out: 9 A True [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] [EOS]
h1_out: 2 2 2 [EOS] [EOS]
h2_out: [EOS] [EOS] [EOS] [EOS] [EOS]
h3_out: False False False False [EOS]
h4_out: 
h_out: False False False False [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] A [EOS]
h1_out: A 7 [EOS] False [EOS]
h2_out: True True [EOS] False [EOS]
h3_out: False False A True [EOS]
h4_out: 
h_out: False False A True [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] 

1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] [EOS] A [EOS] False [EOS] [EOS] [EOS] False [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS]
h1_out: 7 7 A [EOS] [EOS] False [EOS] False [EOS] [EOS] [EOS] False [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 0 [EOS] [EOS] [EOS] 4 [EOS] [EOS] [EOS] [EOS] False [EOS] False [EOS] [EOS] [EOS] False [EOS] 4 [EOS] [EOS] [EOS] [EOS] [EOS] 2 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [

1th highest logit:
h0_out: [EOS] A [EOS] False [EOS]
h1_out: 0-7 False [EOS] False [EOS]
h2_out: A A [EOS] A [EOS]
h3_out: A True A True [EOS]
h4_out: 
h_out: A True A True [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS]
h1_out: 9 0-6 False [EOS]
h2_out: 9 A A [EOS]
h3_out: 7 A True [EOS]
h4_out: 
h_out: 7 A True [EOS]
1th highest logit:
h0_out: [EOS]
h1_out: [EOS]
h2_out: [EOS]
h3_out: [EOS]
h4_out: 
h_out: [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS]
h1_out: [EOS] False [EOS]
h2_out: [EOS] A [EOS]
h3_out: A True [EOS]
h4_out: 
h_out: A True [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] A [EOS]
h1_out: 5 5 [EOS] [EOS] False [EOS]
h2_out: 5 [EOS] [EOS] [EOS] False [EOS]
h3_out: False False False A True [EOS]
h4_out: 
h_out: False False False A True [EOS]
1th highest logit:
h0_out: [EOS] [EOS] [EOS] [EOS] [EOS]
h1_out: 4 4 0 [EOS] [EOS]
h2_out: False [EOS] [EOS] [EOS] [EOS]
h3_out: True False False False [EOS]
h4_out: 
h_out: True False False False [EOS]
1th hi

In [ ]:
for idx in range(1,100):
    model.print_logit_progression(test_set[idx], rank=2)

2th highest logit:
h0_out: [EOS] A A [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 1 A [EOS] [EOS] [EOS] [EOS] 6 9 . [EOS] [EOS] [EOS] [EOS] A A 9 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 1 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] A A A [EOS] [EOS] [EOS] [EOS] [EOS] 3 [EOS] 6 6 6 8 A [EOS] [EOS] [EOS] 1 3 1 [EOS] . [EOS] [EOS] 6 . [EOS] A 7 0 [EOS] . A A A 1 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 1 A [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 9 9 [EOS] [EOS] [EOS] [EOS] A A 9 A A 9 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] 9
h1_out: 2 A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A 1 1 A A A A A A A A A 2 2 A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A 2 A A A A A 2 A A A A A A A A A A A A
h2_out: A A A A A A . . A A A . . . [EOS] [EOS] . . . A A A [EOS] [EOS] . . A A [EOS] . . A . . . A A . [EOS] A . . . . A [EOS] A . . . . A . . . . . . A A A [EOS] A A . . . . . . A . . . . . . A [EOS] . . . . . A A A . . [EOS] A . .

2th highest logit:
h0_out: [EOS] . 6 [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] [EOS] A A [EOS] [EOS] [EOS] [EOS] A A A [EOS] [EOS] [EOS] [EOS] . . 6 A A [EOS] [EOS] [EOS] A A A 6 A A A A A A A 6 . A A [EOS] [EOS] 8 A 1 A [EOS] . . 9 . . [EOS] 6 [EOS] 8 A 2 [EOS] A A A A A A [EOS] [EOS] [EOS] A A A 6 . 3 3 A A A A A A A . [EOS] [EOS] A A A A A [EOS] [EOS] A 0-0 0-0 6 A A A 2 A A A 8 1 8
h1_out: 2 A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A 6 A A 8 A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A 0 A A A A A 8
h2_out: . . A A A A A A A A A . . A A A A A . . . A A A . . . . . . A . . . . [EOS] . . A . . . . A A . A A A A A . . . . . A . . . . . . . . . A A . . . A . A . A . . A [EOS] . . . [EOS] A . A A . . . . A A . . A . A A . . . . . . . . A [EOS] . . . .
h3_out: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [ ]:
def generate_multi_rank_logit_progression(model, test_set, n_ranks, n_samples=100):
    token_dict = []
    
    for seq_idx in tqdm(range(n_samples)):
        entry = {}
        
        # Add label
        entry["label"] = model.decode_tensors(test_set[seq_idx]['labels']).replace("[MASK] ", "")
        
        # Process for each rank
        for rank in range(1, n_ranks + 1):
            rank_result = model.print_logit_progression(test_set[seq_idx], rank=rank)
            
            for k, v in rank_result.items():
                entry[f"r{rank}_{k}"] = "".join(v).split("[EOS]")[0]
        
        token_dict.append(entry)
    
    # Convert to DataFrame
    df = pd.DataFrame(token_dict)
    
    return df

In [ ]:
# Usage
n_ranks = 10  # Set this to the number of ranks you want to process
result_df = generate_multi_rank_logit_progression(model, test_set, n_ranks)

In [ ]:
result_df.label.iloc[0]

In [ ]:
result_df.r1_h_out.iloc[0]

In [ ]:
result_df.r2_h_out.iloc[0]

In [ ]:
result_df.r3_h_out.iloc[0]

In [ ]:
for col in result_df.columns.drop('label'):
    edit_distances[f'{col}_edit_distance'] = result_df.apply(lambda row: Levenshtein.distance(row['label'], row[col]), axis=1)

# Display the first few rows of the new DataFrame
print(edit_distances.head())



In [ ]:
avg_distances = edit_distances.mean().sort_values(ascending=False)

# Create a DataFrame with the average distances
avg_distances_df = pd.DataFrame({
    'Column': avg_distances.index,
    'Average Edit Distance': avg_distances.values
})

# Display the average distances
print("Average Edit Distances:")
print(avg_distances_df)

